In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [4]:
data = pd.read_hdf('data/train.h5')

In [5]:
ids = np.unique(data.id.values)
np.random.shuffle(ids)
cutoff = int(len(ids) * 0.8)
train_ids = ids[:cutoff]
test_ids = ids[cutoff:]

In [6]:
train = data[[row_id in train_ids for row_id in data.id]]
test = data[[row_id in test_ids for row_id in data.id]]

In [7]:
X_train = train.drop('y', axis=1)
X_test = test.drop('y', axis=1)

In [8]:
y_train = train['y']
y_test = test['y']

In [8]:
X_train.describe()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_35,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44
count,1.371207e+06,1.371207e+06,1.317120e+06,1.310691e+06,1.067589e+06,1.256187e+06,1.061032e+06,1.352508e+06,8.134230e+05,1.088758e+06,...,1.368850e+06,1.369262e+06,1.355787e+06,1.355787e+06,1.354936e+06,1.369433e+06,1.335923e+06,1.354936e+06,1.367294e+06,1.181567e+06
mean,1.087683e+03,9.440066e+02,-5.633810e+00,7.817226e+11,-4.480250e-01,-7.873349e-02,2.214370e+01,-1.758670e-02,-7.234223e+08,-8.905497e-02,...,-9.895878e-02,-8.131217e-02,-9.159923e-02,-8.341625e-02,-7.598376e-02,5.047655e-02,5.379525e-03,-1.752768e-02,-9.759589e-01,3.426999e-04
std,6.342526e+02,5.207082e+02,2.781022e+02,7.587538e+13,7.218387e+01,2.150803e+01,1.026413e+03,2.476652e-01,8.449003e+10,2.724590e+00,...,5.558925e-01,6.178950e-01,2.479410e-01,2.373510e-01,2.269784e-01,3.110656e-01,1.134372e-01,2.138127e-01,9.610019e-01,2.999181e-02
min,0.000000e+00,0.000000e+00,-2.017497e+04,-7.375435e-02,-9.848880e+03,-4.186175e+03,-8.551914e+03,-2.344957e+00,-1.043737e+13,-3.828444e+01,...,-1.377945e+00,-1.684077e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-5.250904e-01,-4.449529e-01,-1.000000e+00,-2.000000e+00,-1.265686e-01
25%,5.420000e+02,5.000000e+02,-1.430992e-01,-2.927750e-02,-5.993479e-02,-1.679812e-01,-1.047486e-01,-1.953982e-01,-2.041078e-01,-2.352718e-01,...,-4.219143e-01,-4.053459e-01,-5.258324e-04,-2.308571e-04,-3.314814e-05,-1.496765e-01,-7.345026e-02,-5.892644e-15,-2.000000e+00,-1.994037e-02
50%,1.085000e+03,9.540000e+02,-7.291897e-04,6.491869e-03,2.201143e-02,3.620134e-04,1.045790e-02,-3.666383e-02,-8.130231e-03,-3.437910e-02,...,-1.126339e-01,-8.421298e-02,-4.539158e-12,-2.488125e-13,-6.364897e-16,-1.183149e-02,3.540279e-04,0.000000e+00,-6.804921e-01,-6.244372e-05
75%,1.659000e+03,1.402000e+03,1.199108e-01,1.144038e-01,1.997509e-01,3.024452e-01,1.543160e-01,1.336145e-01,1.745031e-01,1.728605e-01,...,1.635433e-01,1.962546e-01,-1.316226e-40,0.000000e+00,0.000000e+00,1.740852e-01,7.887742e-02,0.000000e+00,-6.846776e-08,2.034125e-02
max,2.158000e+03,1.812000e+03,3.252527e+03,1.068448e+16,3.823001e+03,1.239737e+03,6.785965e+04,1.378195e+00,5.203165e+02,7.677125e+01,...,1.464272e+01,4.957758e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.569265e+00,6.844833e-01,1.000000e+00,0.000000e+00,1.435858e-01


In [9]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [ ]:
structures = [('technical_34', 'technical_22')]
nan_structures_split = []


In [10]:
len(X_train.values)

1358577

In [11]:
clf = RandomForestRegressor()
clf.fit(X_train.values, y_train.values)
clf.score(X_test.values, y_test.values)

-0.14410489174158214

In [ ]:
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train.values[:20000], y_train.values[:20000])
xgb_clf.score(X_test.values[:20000], y_test.values[:20000])

In [ ]:
et_clf = ExtraTreesRegressor(n_estimators=100, max_depth=4, n_jobs=-1, random_state=17, verbose=0)
et_clf.fit(X_train.values, y_train.values)
et_clf.score(X_test.values, y_test.values)

0.00062695955961855621